In [20]:
import boto3
import pandas as pd
import glob
import os
import json
import xmltodict
import sys
from collections import Counter

In [2]:
fn = glob.glob(os.path.join('.', '*final.json'))[0]
fr = glob.glob(os.path.join('results1', '*final.json'))[0]


In [3]:
fn

'./created_2019-Jan-23_18_34_43_final.json'

In [4]:
fr

'results1/created_2019-Jan-23_18_34_43_final.json'

In [14]:
with open(fn, 'r') as handle:
    hitlist = json.load(handle)
with open(fr, 'r') as handle:
    resultlist = json.load(handle)

In [15]:
hit2images = {}
for hit in hitlist:
    hit2images[hit['HIT']['HITId']] = hit['Images']

In [16]:
resultlist[0]

{'Assignments': [{'Answers': {'0': {'inputbox_objname-0': 'cereal',
     'other_reasons-0': None},
    '1': {'inputbox_objname-1': 'gazebo', 'other_reasons-1': None},
    '2': {'inputbox_objname-2': 'soup', 'other_reasons-2': None},
    '3': {'inputbox_objname-3': 'dog', 'other_reasons-3': None},
    '4': {'inputbox_objname-4': 'acke', 'other_reasons-4': None},
    '5': {'inputbox_objname-5': 'dog', 'other_reasons-5': None},
    '6': {'inputbox_objname-6': 'cookie', 'other_reasons-6': None},
    '7': {'inputbox_objname-7': 'human', 'other_reasons-7': None},
    '8': {'inputbox_objname-8': 'human', 'other_reasons-8': None},
    '9': {'inputbox_objname-9': 'scissors', 'other_reasons-9': None},
    's': {'comments': None}},
   'AssignmentId': '323Q6SJS8KL88NP8ZJY85NHN02IHF0',
   'WorkerId': 'A1G70F4VJOW46H'},
  {'Answers': {'0': {'inputbox_objname-0': 'cereal', 'other_reasons-0': None},
    '1': {'inputbox_objname-1': 'building', 'other_reasons-1': None},
    '2': {'inputbox_objname-2': '

In [34]:
hit2answers = {}
for assignm in resultlist:
    hit2answers[assignm['HITId']] = {ix:Counter() for ix in range(10)}
    for a in assignm['Assignments']:
        #print(a)
        answers = a['Answers']
        for ix in range(10):
            name = answers[str(ix)]['inputbox_objname-'+str(ix)]
            if name:
                name = name.lower()
                hit2answers[assignm['HITId']][ix][name] += 1
                
            


In [35]:
hit2answers

{'31MCUE39BLRW5VGA5GAMUUK44JVG37': {0: Counter({'rice': 9, 'soia beans': 1}),
  1: Counter({'bread': 1, 'cake': 1, 'pancake': 5, 'pancakes': 1}),
  2: Counter({'onion': 1,
           'rhubarb': 3,
           'spinach': 1,
           'vegetable stalks': 1,
           'vegetables': 1,
           'vegtable': 1}),
  3: Counter({'burger': 5, 'sandwich': 4, 'sandwitch': 1}),
  4: Counter({'house': 8, 'shelter': 1, 'wall': 1}),
  5: Counter({'blender': 3,
           'blender cup': 1,
           'blender/mixer container': 1,
           'juice and jar': 1,
           'juicer': 3,
           'mixer': 1}),
  6: Counter({'bread and oven': 1,
           'micro oven': 1,
           'oven': 2,
           'paddle': 1,
           'peel': 1,
           'shovel': 1}),
  7: Counter({'dog': 9, 'pug': 1}),
  8: Counter({'building': 3,
           'bulding': 1,
           'house': 2,
           'large apartment house/inn.': 1,
           'men': 1,
           'palace': 2}),
  9: Counter({'artist': 1,
         

In [36]:
hit2images

{'31MCUE39BLRW5VGA5GAMUUK44JVG37': {'0': ['2384848',
   '1304092',
   'ambiguous_category',
   'foodstuff.n.02',
   'http://www.coli.uni-saarland.de/~carina/object_naming/amt_images/2384848_1304092_ambiguous_category.png'],
  '1': ['2415689',
   '3313513',
   'ambiguous_synset',
   'baked_goods.n.01',
   'http://www.coli.uni-saarland.de/~carina/object_naming/amt_images/2415689_3313513_ambiguous_synset.png'],
  '2': ['2361816',
   '2220050',
   'ambiguous_synset',
   'vegetable.n.01',
   'http://www.coli.uni-saarland.de/~carina/object_naming/amt_images/2361816_2220050_ambiguous_synset.png'],
  '3': ['2359393',
   '793716',
   'singleton_category',
   'dish.n.02',
   'http://www.coli.uni-saarland.de/~carina/object_naming/amt_images/2359393_793716_singleton_category.png'],
  '4': ['2372994',
   '1749119',
   'ambiguous_synset',
   'house.n.01',
   'http://www.coli.uni-saarland.de/~carina/object_naming/amt_images/2372994_1749119_ambiguous_synset.png'],
  '5': ['2350930',
   '1830045',
   '

In [44]:

outdf = []
for hitid in hit2images:
    for imid in hit2images[hitid]:
        rl = [hitid,imid]
        for x in hit2images[hitid][imid]:
            rl.append(x)
        rl.append(hit2answers[hitid][int(imid)])
        outdf.append(rl)

In [45]:
outdf = pd.DataFrame(outdf,columns=['hitid','img_in_hit','vg_img_id','vg_object_id','cat','synset','url','responses'])

In [46]:
outdf

,hitid,img_in_hit,vg_img_id,vg_object_id,cat,synset,url,responses
0,382GHPVPHTWGTFNQAF8LL5N1EEM438,0,884,1637760,singleton_category,foodstuff.n.02,http://www.coli.uni-saarland.de/~carina/object...,"{'cereal': 3, 'calox chocos': 1, 'drink': 1, '..."
1,382GHPVPHTWGTFNQAF8LL5N1EEM438,1,3947,1070734,ambiguous_category,outbuilding.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'gazebo': 1, 'building': 2, 'house': 3, 'roof..."
2,382GHPVPHTWGTFNQAF8LL5N1EEM438,2,2397003,1193594,ambiguous_category,dish.n.02,http://www.coli.uni-saarland.de/~carina/object...,"{'soup': 3, 'vegetable stew': 1, 'dish': 1, 'c..."
3,382GHPVPHTWGTFNQAF8LL5N1EEM438,3,2346351,2280590,singleton_category,dog.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'dog': 7, 'shih-poo': 2, 'dady bar': 1}"
4,382GHPVPHTWGTFNQAF8LL5N1EEM438,4,2363996,3339443,ambiguous_synset,baked_goods.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'acke': 1, 'cake': 4, 'desert': 1, 'stollen':..."
5,382GHPVPHTWGTFNQAF8LL5N1EEM438,5,2357137,814606,ambiguous_synset,dog.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'dog': 9, 'standard boodle': 1}"
6,382GHPVPHTWGTFNQAF8LL5N1EEM438,6,2374569,586596,singleton_category,baked_goods.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'cookie': 6, 'biscuit': 2, 'statue': 1}"
7,382GHPVPHTWGTFNQAF8LL5N1EEM438,7,2414319,296869,ambiguous_synset,woman.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'human': 1, 'woman on phone': 1, 'mobile': 2,..."
8,382GHPVPHTWGTFNQAF8LL5N1EEM438,8,2338246,3491340,ambiguous_category,athlete.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'human': 1, 'tennis player': 4, 'man': 2, 'fe..."
9,382GHPVPHTWGTFNQAF8LL5N1EEM438,9,2355516,1850502,ambiguous_synset,tool.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'scissors': 5, 'scissor': 4, 'sizzers': 1}"


In [48]:
outdf.to_csv("prepilot_processed_results.csv",index=False)